In [2]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())


2023-06-21 16:20:37,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 16:20:37,311 INFO sqlalchemy.engine.Engine select 'hello world'
2023-06-21 16:20:37,312 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()
[('hello world',)]
2023-06-21 16:20:37,312 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-06-21 16:21:00,571 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 16:21:00,573 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-06-21 16:21:00,573 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()
2023-06-21 16:21:00,574 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-06-21 16:21:00,574 INFO sqlalchemy.engine.Engine [cached since 18.43s ago] [(1, 1), (2, 4)]
2023-06-21 16:21:00,574 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES(:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-06-21 17:16:33,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 17:16:33,219 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES(?, ?)
2023-06-21 17:16:33,220 INFO sqlalchemy.engine.Engine [cached since 3063s ago] [(6, 8), (9, 10)]
2023-06-21 17:16:33,220 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2023-06-21 16:28:33,831 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 16:28:33,832 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-06-21 16:28:33,832 INFO sqlalchemy.engine.Engine [cached since 21.18s ago] ()
x: 1, y: 1
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2023-06-21 16:28:33,833 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table where y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2023-06-21 17:26:30,208 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 17:26:30,209 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table where y > ?
2023-06-21 17:26:30,209 INFO sqlalchemy.engine.Engine [cached since 541.6s ago] (2,)
x: 2, y: 4
x: 6, y: 8
x: 9, y: 11
x: 6, y: 8
x: 9, y: 11
2023-06-21 17:26:30,211 INFO sqlalchemy.engine.Engine ROLLBACK


In [14]:
from sqlalchemy.orm import Session
with Session(engine.connect()) as sess:
    result = sess.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}]
    )
    sess.commit()

2023-06-21 17:26:25,170 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-21 17:26:25,171 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2023-06-21 17:26:25,172 INFO sqlalchemy.engine.Engine [generated in 0.00075s] [(11, 9), (15, 13)]
2023-06-21 17:26:25,172 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [4]:
from sqlalchemy import Table, Column, String, Integer

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [8]:
user_table.c.name
user_table.c.keys()
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [11]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [14]:
metadata_obj.create_all(engine)

2023-06-23 16:02:33,732 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 16:02:33,732 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-23 16:02:33,733 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 16:02:33,734 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-23 16:02:33,734 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 16:02:33,735 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-06-23 16:02:33,735 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 16:02:33,736 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-06-23 16:02:33,736 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 16:02:33,737 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-06-23 16:02:33,738 INFO sqlalchemy.engine.Engine [no key 0.00072s] ()
2023-06-23 16:02:33,739 INFO sqlalchemy.engine.Engine 
C

In [1]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

Base.metadata

MetaData()

In [5]:
from typing import List, Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import String

class User(Base):
    __tablename__ = "orm_user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]] = mapped_column(String)

    addresses: Mapped[List[str]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

        
class Address(Base):
    __tablename__ = "orm_address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

TypeError: 'type' object does not support item assignment

In [ ]:
Base.metadata.create_all(engine)